# Needed imports and utility

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from comtypes.client import CreateObject

import ansys.speos.core as core
from ansys.speos.core.speos import Speos
import ansys.speos.workflow as workflow

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")

def print_message(msg):
    print(core.protobuf_message_to_str(msg))

def open_file(file):
    dpf_instance = None
    if file.endswith("xmp") or file.endswith("XMP"):
        dpf_instance = CreateObject("XMPViewer.Application")
        dpf_instance.OpenFile(file)
        res = dpf_instance.ExportXMPImage(file+".png", 1)
        if res:
            img = mpimg.imread(file+".png")
            plt.imshow(img)
            plt.show()
    if file.endswith("hdr") or file.endswith("HDR"):
        dpf_instance = CreateObject("HDRIViewer.Application")
        dpf_instance.OpenFile(file)
        dpf_instance.Show(1)
    return dpf_instance

# Create connection with speos rpc server

In [ ]:
speos = Speos(host="localhost", port=50051)
workflow.clean_all_dbs(speos.client)

# Create SpeosSimulationUpdate

In [ ]:
sim_file = os.path.join(tests_data_path, "student", "Prism", "Direct.1.speos")
ssu = workflow.SpeosSimulationUpdate(speos=speos, file_name=sim_file)
print(ssu.status)
print(ssu.scene)

# New Irradiance sensor
## Create two IrradianceSensorParameters

In [ ]:
sensors = []

irradiance_sensor = workflow.IrradianceSensorParameters() # use default parameters
sensors.append(irradiance_sensor)

irradiance_sensor_new = irradiance_sensor.copy() # assign values to parameters
irradiance_sensor_new.name = "new_sensor"
irradiance_sensor_new.integration_type = core.SensorTemplateFactory.IlluminanceType.Planar
irradiance_sensor_new.type = core.SensorTemplateFactory.Type.Spectral
irradiance_sensor_new.wavelengths_start = 380
irradiance_sensor_new.wavelengths_end = 780
irradiance_sensor_new.wavelengths_sampling = 41
irradiance_sensor_new.x_range_start = -40
irradiance_sensor_new.x_range_end = 40
irradiance_sensor_new.x_range_sampling = 400
irradiance_sensor_new.y_range_start = -5
irradiance_sensor_new.y_range_end = 5
irradiance_sensor_new.y_range_sampling = 100
sensors.append(irradiance_sensor_new)


# Create IrradianceSensorProperties

In [ ]:
irradiance_props = workflow.IrradianceSensorProperties()
irradiance_props.origin = [921.36, 0, 388.72]
irradiance_props.x_vector = [0.388721970152395, 0, -0.921355105223193]
irradiance_props.y_vector = [0.0, 1.0, 0.0]
irradiance_props.z_vector = [0.921355105223193, 0.0, 0.388721970152395]

# Add Irradiance sensors to simulation

In [ ]:
for sensor in sensors:
    ssu.add_irradiance_sensor(sensor, irradiance_props)

# Compute simulation

In [ ]:
job_link = ssu.compute("first_job")
print_message(job_link.get_results())
open_file(job_link.get_results().results[2].path)
open_file(job_link.get_results().results[1].path)

# Modify the irradiance sensor

In [ ]:
irradiance_sensor_update = irradiance_sensor_new.copy()

irradiance_props_update = workflow.IrradianceSensorProperties()
irradiance_props_update.origin = [1000, 0, 400]
irradiance_props_update.x_vector = [0.388721970152395, 0, -0.921355105223193]
irradiance_props_update.y_vector = [0.0, 1.0, 0.0]
irradiance_props_update.z_vector = [0.921355105223193, 0.0, 0.388721970152395]

ssu.update_sensor(irradiance_sensor_update, irradiance_props_update)

# Compute the modified simulation

In [ ]:
job_link = ssu.compute("second_job")
print_message(job_link.get_results())
open_file(job_link.get_results().results[2].path)

# Compute the modified simulation using GPU

In [ ]:
job_link = ssu.compute("second_job", compute_type="gpu")
print_message(job_link.get_results())
open_file(job_link.get_results().results[2].path)

# Clean after usage

In [ ]:
job_link.delete()
ssu.close()
print(ssu.status)